In [ ]:
import os, sys

BASE_PATH = ''

os.chdir(BASE_PATH)

In [2]:
import faiss
import yaml
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
with open('data/mkb2descr_new.yaml', 'r') as f:
    mkb2descr = yaml.safe_load(f)
descr2mkb = {f'{v} - {k}': k for k, v in mkb2descr.items()}
len(mkb2descr), len(descr2mkb)

(17762, 17762)

In [4]:
oper_train_df = pd.read_parquet('data/oper_train_df_v2.parquet')
oper_test_df = pd.read_parquet('data/oper_test_df_v2.parquet')
print(oper_train_df.shape, oper_test_df.shape)
oper_train_df.head()

(3000, 5) (500, 5)


,text,text_labels,labled_entities,mkb_texts,mkb_codes
0,Эндоцервикоз. Хронический сальпингит и оофорит...,<mkb code=N88.8>Эндоцервикоз</mkb>. <mkb code=...,<mkb>Эндоцервикоз</mkb>. <mkb>Хронический саль...,"[Эндоцервикоз, Хронический сальпингит и оофори...","[N88.8, N70.1, N73.6]"
1,"Ожирение 3 степени, смешанного генеза","<mkb code=E66.9>Ожирение </mkb>3 степени, смеш...","<mkb>Ожирение </mkb>3 степени, смешанного генеза",[Ожирение ],[E66.9]
2,Полип цервикального канала. Эрозия шейки матки.,<mkb code=N84.1>Полип цервикального канала.</m...,<mkb>Полип цервикального канала.</mkb> <mkb>Эр...,"[Полип цервикального канала., Эрозия шейки матки]","[N84.1, N87]"
3,Гастроэнтероколит. предположительно инф. этиол...,<mkb code=A09>Гастроэнтероколит. предположител...,<mkb>Гастроэнтероколит. предположительно инф. ...,[Гастроэнтероколит. предположительно инф. этио...,[A09]
4,острый катаральный фронтит,<mkb code=J01.1>острый катаральный фронтит</mkb>,<mkb>острый катаральный фронтит</mkb>,[острый катаральный фронтит],[J01.1]


In [5]:
oper_mkb_train_df = oper_train_df[['mkb_texts', 'mkb_codes']].explode(['mkb_texts', 'mkb_codes'])
oper_mkb_train_df.head()

,mkb_texts,mkb_codes
0,Эндоцервикоз,N88.8
0,Хронический сальпингит и оофорит,N70.1
0,Спаечный процесс в малом тазу,N73.6
1,Ожирение,E66.9
2,Полип цервикального канала.,N84.1


In [6]:
oper_mkb_test_df = oper_test_df[['mkb_texts', 'mkb_codes']].explode(['mkb_texts', 'mkb_codes'])
oper_mkb_test_df.head()

,mkb_texts,mkb_codes
0,Аднексит,N70.9
0,Кисты экзоцервикса,N88.8
1,Гиперандрогени,E28.1
1,Вагинит,N76.0
1,сальпингоофорит,N70.1


In [7]:
icd_retrieval_df = pd.read_parquet('data/mkb_descr_and_emb_df_bergamot.parquet')
icd_retrieval_df.head()

,text,mkb,emb
0,Коды для использования при отсутствии диагноза...,00,"[0.20321438, 0.7739176, 0.43605193, 1.1479815,..."
1,Холера - A00,A00,"[0.6391061, 0.5850919, 0.58136743, 1.2119223, ..."
2,КИШЕЧНЫЕ ИНФЕКЦИИ (A00-A09) - A00-A09,A00-A09,"[0.8070125, 0.7129453, 0.41063482, 1.2905184, ..."
3,НЕКОТОРЫЕ ИНФЕКЦИОННЫЕ И ПАРАЗИТАРНЫЕ БОЛЕЗНИ ...,A00-B99,"[0.7257797, 0.70322335, 0.54525924, 1.0922508,..."
4,"Холера, вызванная холерным вибрионом 01, биова...",A00.0,"[0.81637913, 0.76124996, 0.8273382, 1.3087286,..."


In [8]:
train_mkb_texts_and_emb_df = pd.read_parquet('data/train_mkb_texts_and_emb_df_bergamot.parquet')
train_mkb_text2emb = train_mkb_texts_and_emb_df.set_index('text')['emb'].to_dict()
print(len(train_mkb_text2emb))
train_mkb_texts_and_emb_df.head()

4517


,text,text_without_abbr,mkb,emb
0,Эндоцервикоз,Эндоцервикоз,N88.8,"[1.0646714, 0.55818516, 0.25616288, 1.2985225,..."
1,Хронический сальпингит и оофорит,Хронический сальпингит и оофорит,N70.1,"[1.0741761, 0.56112343, 0.5924396, 1.4484705, ..."
2,Спаечный процесс в малом тазу,Спаечный процесс в малом тазу,N73.6,"[1.0086013, 0.6514759, 0.229128, 0.9506954, -0..."
3,Ожирение,Ожирение,E66.9,"[0.9568775, 0.5229445, 0.51183635, 1.3658271, ..."
4,Полип цервикального канала.,Полип цервикального канала.,N84.1,"[1.0374979, 0.5656024, 0.47970936, 1.5457002, ..."


In [9]:
test_mkb_texts_and_emb_df = pd.read_parquet('data/test_mkb_texts_and_emb_df_bergamot.parquet')
test_mkb_text2emb = test_mkb_texts_and_emb_df.set_index('text')['emb'].to_dict()
print(len(test_mkb_text2emb))
test_mkb_texts_and_emb_df.head()

1116


,text,text_without_abbr,mkb,emb
0,Аднексит,Аднексит,N70.9,"[0.84549487, 0.6559077, 0.57338065, 0.97752774..."
1,Кисты экзоцервикса,Кисты экзоцервикса,N88.8,"[1.0207729, 0.64380205, 0.14841671, 1.4028211,..."
2,Гиперандрогени,Гиперандрогени,E28.1,"[1.2520971, 0.56747735, 0.34002924, 1.31798, -..."
3,Вагинит,Вагинит,N76.0,"[1.1868458, 0.5189025, 0.28673723, 1.3597, -0...."
4,сальпингоофорит,сальпингоофорит,N70.1,"[1.1721609, 0.7023772, 0.75886655, 1.4234688, ..."


In [10]:
train_and_icd_retrieval_df = pd.concat([
    train_mkb_texts_and_emb_df[['text', 'mkb', 'emb']],
    icd_retrieval_df[['text', 'mkb', 'emb']]
])
train_and_icd_retrieval_df = train_and_icd_retrieval_df.drop_duplicates(['text']).reset_index(drop=True)
print(train_and_icd_retrieval_df.shape)

(22279, 3)


In [11]:
train_and_icd_retrieval_df

,text,mkb,emb
0,Эндоцервикоз,N88.8,"[1.0646714, 0.55818516, 0.25616288, 1.2985225,..."
1,Хронический сальпингит и оофорит,N70.1,"[1.0741761, 0.56112343, 0.5924396, 1.4484705, ..."
2,Спаечный процесс в малом тазу,N73.6,"[1.0086013, 0.6514759, 0.229128, 0.9506954, -0..."
3,Ожирение,E66.9,"[0.9568775, 0.5229445, 0.51183635, 1.3658271, ..."
4,Полип цервикального канала.,N84.1,"[1.0374979, 0.5656024, 0.47970936, 1.5457002, ..."
...,...,...,...
22274,Зависимость от почечного диализа - Z99.2,Z99.2,"[0.5967582, 0.60067123, 0.5826726, 1.0184367, ..."
22275,Зависимость от кресла на колесах - Z99.3,Z99.3,"[1.1073288, 0.5780251, 0.21158911, 0.9499818, ..."
22276,Зависимость от искусственного сердца - Z99.4,Z99.4,"[0.81013167, 0.6914784, 0.55219746, 0.90709496..."
22277,Зависимость от других вспомогательных механизм...,Z99.8,"[0.70633715, 0.76928276, 0.41433388, 0.768888,..."


In [12]:
icd_retrieval_embeddings = np.vstack(icd_retrieval_df['emb'].to_list())
icd_retrieval_dimension = icd_retrieval_embeddings.shape[1]
icd_retrieval_index = faiss.IndexFlatL2(icd_retrieval_dimension)
icd_retrieval_index.add(icd_retrieval_embeddings)

In [13]:
train_and_icd_retrieval_embeddings = np.vstack(train_and_icd_retrieval_df['emb'].to_list())
train_and_icd_retrieval_dimension = train_and_icd_retrieval_embeddings.shape[1]
train_and_icd_retrieval_index = faiss.IndexFlatL2(train_and_icd_retrieval_dimension)
train_and_icd_retrieval_index.add(train_and_icd_retrieval_embeddings)

In [14]:
def create_oper_retrieval_df(
        initial_df,
        initial_embs,
        retrieval_df,
        retrieval_index,
        n
    ):
    oper_retrieval_ds = []
    for mkb_text, mkb in tqdm(initial_df.to_records(index=False)):
        if isinstance(mkb_text, str):
            text_emb = initial_embs[mkb_text]
        else:
            continue
        _, idxs = retrieval_index.search(np.expand_dims(text_emb, axis=0), n)
        founded_records = retrieval_df.iloc[idxs[0].tolist()]
        
        if mkb not in founded_records['mkb'].to_list():
            mkb = founded_records['mkb'].to_list()[0]

        founded_records = founded_records.sample(frac=1)
        founded_descr = founded_records['text'].to_list()
        founded_mkbs = founded_records['mkb'].to_list()
        label = founded_mkbs.index(mkb)

        oper_retrieval_ds.append({
            'mkb_text': mkb_text,
            'founded_mkb_descriptions': founded_descr,
            'label': label
        })

    return pd.DataFrame(oper_retrieval_ds)

In [15]:
train_icd_retrieval_df = create_oper_retrieval_df(
    oper_mkb_train_df, train_mkb_text2emb,
    icd_retrieval_df, icd_retrieval_index,
    15
)
train_icd_retrieval_df['label'].value_counts()

  0%|          | 0/8383 [00:00<?, ?it/s]

label
0     576
7     573
5     571
3     569
13    568
12    567
8     566
1     560
2     558
4     552
10    551
9     548
11    539
14    538
6     518
Name: count, dtype: int64

In [16]:
train_tr_and_icd_retrieval_df = create_oper_retrieval_df(
    oper_mkb_train_df, train_mkb_text2emb,
    train_and_icd_retrieval_df, train_and_icd_retrieval_index,
    15
)
train_tr_and_icd_retrieval_df['label'].value_counts()

  0%|          | 0/8383 [00:00<?, ?it/s]

label
0     3146
1     1500
2      923
3      605
4      441
5      377
6      283
7      224
8      199
9      142
10     132
11     120
12      98
13      86
14      78
Name: count, dtype: int64

In [17]:
test_icd_retrieval_df = create_oper_retrieval_df(
    oper_mkb_test_df, test_mkb_text2emb,
    icd_retrieval_df, icd_retrieval_index,
    15
)
test_icd_retrieval_df['label'].value_counts()

  0%|          | 0/1513 [00:00<?, ?it/s]

label
5     122
4     115
1     112
12    108
14    108
11    106
3     103
13    101
2      96
7      96
6      95
0      94
8      87
10     87
9      81
Name: count, dtype: int64

In [18]:
test_tr_and_icd_retrieval_df = create_oper_retrieval_df(
    oper_mkb_test_df, test_mkb_text2emb,
    train_and_icd_retrieval_df, train_and_icd_retrieval_index,
    15
)
test_tr_and_icd_retrieval_df['label'].value_counts()

  0%|          | 0/1513 [00:00<?, ?it/s]

label
0     539
1     261
2     175
3     115
4      82
5      59
6      45
7      43
8      42
11     37
10     27
9      27
12     21
14     20
13     18
Name: count, dtype: int64

In [19]:
#train_icd_retrieval_df.to_parquet('data/retrieval_ds/train_icd_retrieval_df_stratified.parquet')
#test_icd_retrieval_df.to_parquet('data/retrieval_ds/test_icd_retrieval_df_stratified.parquet')

train_icd_retrieval_df.to_parquet('data/retrieval_ds/train_icd_retrieval_df_stratified_bergamot.parquet')
test_icd_retrieval_df.to_parquet('data/retrieval_ds/test_icd_retrieval_df_stratified_bergamot.parquet')

In [20]:
#train_tr_and_icd_retrieval_df.to_parquet('data/retrieval_ds/train_tr_and_icd_retrieval_df_stratified.parquet')
#test_tr_and_icd_retrieval_df.to_parquet('data/retrieval_ds/test_tr_and_icd_retrieval_df_stratified.parquet')

train_icd_retrieval_df.to_parquet('data/retrieval_ds/train_icd_retrieval_df_stratified_bergamot.parquet')
test_icd_retrieval_df.to_parquet('data/retrieval_ds/test_icd_retrieval_df_stratified_bergamot.parquet')